<a href="https://colab.research.google.com/github/Ra0k/alexa_skill_bilingual_audiobook/blob/main/bilingual_reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download Translator

In [2]:
%%bash
#pip install fastBPE regex requests sacremoses subword_nmt omegaconf hydra-core
pip install transformers

  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=288f8cb97b0e7d2e83f3da07bfd58e3ad8aa8715dfc8818e9dc9b143cb66b2a6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
mname = "facebook/wmt19-de-en"
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)


In [4]:
input = "Sternschnuppen unten in Kent ich wette, das war Dädalus Diggel."
input_ids = tokenizer.encode(input, return_tensors="pt")
outputs = model.generate(input_ids)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded)

Shooting stars down in Kent I bet that was Daedalus Diggel.


In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 312,778,752 trainable parameters


# Read file

In [2]:
def readFile(filename):
    with open(filename, 'rt') as file:
      text = file.read()
    
    return text

In [4]:
import os
base_path = "/content/drive/My Drive/Colab Notebooks/translator/content/"
book_name = "de_kafka_prozess.txt"
book_name_en = "en_kafka_prozess.txt"
book_name_base, book_name_ext = os.path.splitext(book_name)

sentences = readFile(base_path + book_name).split('\n')


FileNotFoundError: ignored

# Translate Sentences

In [76]:
"There are %d sentences to translate."%len(sentences)

'There are 3608 sentences to translate.'

In [77]:
translations = []
for idx, sentence in enumerate(sentences):
  input_ids = tokenizer.encode(sentence, return_tensors="pt")
  outputs = model.generate(input_ids)
  decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
  translations.append({
      'idx': idx,
      'de': sentence, 
      'en': decoded
  })
  if idx % 100 == 0:
    print('%dth batch is done!'%idx)

0th batch is done!
100th batch is done!
200th batch is done!
300th batch is done!
400th batch is done!
500th batch is done!
600th batch is done!
700th batch is done!
800th batch is done!
900th batch is done!
1000th batch is done!
1100th batch is done!
1200th batch is done!
1300th batch is done!
1400th batch is done!
1500th batch is done!
1600th batch is done!
1700th batch is done!
1800th batch is done!
1900th batch is done!
2000th batch is done!
2100th batch is done!
2200th batch is done!
2300th batch is done!
2400th batch is done!
2500th batch is done!
2600th batch is done!
2700th batch is done!
2800th batch is done!
2900th batch is done!
3000th batch is done!
3100th batch is done!
3200th batch is done!
3300th batch is done!
3400th batch is done!
3500th batch is done!
3600th batch is done!


# Save Translation

In [78]:
with open("%s%s_translated%s"%(base_path, book_name_base, book_name_ext), "w") as f:
  for item in translations:
    f.write("%s|%s|%s\n"%(str(item['idx']), item['de'], item['en']))

# Read the translated and original English files

In [6]:
sentences = readFile(base_path+book_name_base+"_translated"+book_name_ext).split('\n') 

In [7]:
de_sentences = []
en_sentences_translated = []
for sentence in sentences: 
    if "|" in sentence:
        de_sentence = sentence.split("|")[1]
        en_sentence = sentence.split("|")[2]

        de_sentences.append(de_sentence)
        en_sentences_translated.append(en_sentence)        

In [8]:
en_sentences = readFile(base_path + book_name_en).split('\n')

In [9]:
en_sentences[:10]

['The Trial Franz Kafka Chapter One Arrest - Conversation with Mrs. Grubach - Then Miss Bürstner',
 "Someone must have been telling lies about Josef K., he knew he had done nothing wrong but, one morning, he was arrested. Every day at eight in the morning he was brought his breakfast by Mrs. Grubach's cook - Mrs. Grubach was his landlady - but today she didn't come. That had never happened before. K. waited a little while, looked from his pillow at the old woman who lived opposite and who was watching him with an inquisitiveness quite unusual for her, and finally, both hungry and disconcerted, rang the bell. There was immediately a knock at the door and a man entered. He had never seen the man in this house before. He was slim but firmly built, his clothes were black and close-fitting, with many folds and pockets, buckles and buttons and a belt, all of which gave the impression of being very practical but without making it very clear what they were actually for.",
 '"Who are you?" aske

In [10]:
en_sentences_translated[:10]

['The trial, Franz Kafka 1 arrest, conversation with Mrs. Grubach, then Miss Brüstner Someone must have slandered Josef K., because without having done anything wrong he was arrested one morning.',
 "The cook of Mrs. Grubach, his room landlord, who brought him breakfast every day at around eight o'clock in the morning, did not come this time.",
 'This had never happened before.',
 'K. waited a little longer, saw from his pillow the old woman who lived opposite him, and who observed him with a curiosity quite unusual in her, but then, at the same time bewildered and hungry, he rang the bell.',
 'Immediately there was a knock and a man he had never seen in this apartment stepped in.',
 'He was slender and yet solidly built, wearing a fitted black dress, which, similar to travel suits, had various pleats, pockets, buckles, buttons and a belt, and as a result, appeared particularly practical without anyone knowing what it was for.',
 '"Who are you?" K. asked and sat half upright in bed.',


# Preprocessing before fine-tuning Word2Vec vectors

In [11]:
!pip install sentence_splitter

     |████████████████████████████████| 51kB 2.6MB/s 


In [12]:
text = ' '.join([str(elem) for elem in en_sentences]) 

In [13]:
from sentence_splitter import SentenceSplitter, split_text_into_sentences

splitter = SentenceSplitter(language='en')
all_en_sentences = splitter.split(text=text)

In [14]:
all_en_sentences[:10]

['The Trial Franz Kafka Chapter One Arrest - Conversation with Mrs. Grubach - Then Miss Bürstner Someone must have been telling lies about Josef K., he knew he had done nothing wrong but, one morning, he was arrested.',
 "Every day at eight in the morning he was brought his breakfast by Mrs. Grubach's cook - Mrs. Grubach was his landlady - but today she didn't come.",
 'That had never happened before.',
 'K. waited a little while, looked from his pillow at the old woman who lived opposite and who was watching him with an inquisitiveness quite unusual for her, and finally, both hungry and disconcerted, rang the bell.',
 'There was immediately a knock at the door and a man entered.',
 'He had never seen the man in this house before.',
 'He was slim but firmly built, his clothes were black and close-fitting, with many folds and pockets, buckles and buttons and a belt, all of which gave the impression of being very practical but without making it very clear what they were actually for.',
 

In [15]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
#save translated text seperately
# train a language model -en with translated sentences 
import spacy
import re
from nltk.stem import PorterStemmer
porter = PorterStemmer()
min_len = 7
def tokenize(sentence):
    sentence = sentence.strip()
    sentence = sentence.lower()
    sentence = re.sub(r'n\'t', " not", sentence)
    sentence = re.sub(r'\'ll', " will", sentence)
    sentence = re.sub(r'\'s', " is", sentence)     
    sentence = re.sub(r'\'m', " am", sentence)    
    sentence = re.sub(r'\'re', " are", sentence)
    sentence = re.sub(r'\'ve', " have", sentence)
    sentence = sentence.replace('\n', '')  
    sentence = re.sub(r"[^a-z'äüöß]+", " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    tokens = en.tokenizer(sentence)
    if len(tokens)>min_len:
        return [porter.stem(tok.text) for tok in en.tokenizer(sentence) if porter.stem(tok.text) not in stopwords]
    else:
        return [porter.stem(tok.text) for tok in en.tokenizer(sentence)]
en = spacy.load('en_core_web_sm')



# Training Word2Vec vectors

Train a Word2vec using both original and translated english sentences

In [17]:
import gensim
import numpy as np


W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 100
W2V_MIN_COUNT = 2


texts = [tokenize(_text) for _text in np.array(en_sentences_translated+all_en_sentences)] # divided into single sentences

# Train Word Embeddings and save
w2v_model = gensim.models.word2vec.Word2Vec(size=W2V_SIZE, window=W2V_WINDOW,                                                                                  min_count=W2V_MIN_COUNT)
w2v_model.build_vocab(texts)
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size: ", vocab_size)

w2v_model.train(texts, total_examples=len(texts), epochs=W2V_EPOCH)
w2v_model.save('embeddings.txt')

Vocab size:  3145


In [18]:
w2v_model = gensim.models.word2vec.Word2Vec.load('embeddings.txt')

In [19]:
w2v_model.most_similar('mr')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('ms', 0.7201731204986572),
 ('frau', 0.7011095285415649),
 ('apologet', 0.42527681589126587),
 ('nephew', 0.3429846465587616),
 ('herr', 0.3137998878955841),
 ('committe', 0.29427823424339294),
 ('offend', 0.276938259601593),
 ('authoris', 0.26617035269737244),
 ('captain', 0.2608180344104767),
 ('betray', 0.25873416662216187)]

# Matching the sentences

Calculate sentence similarity by comparing avg of word vectors in the sentences

In [20]:
import numpy as np
from scipy import spatial

index2word_set = set(w2v_model.wv.index2word)

def avgFeatureVector(sentence, model, num_features, index2word_set):
    words = tokenize(sentence)
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in set(words):
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])              

    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [21]:
def normalize(arr):
  return arr / max(np.linalg.norm(arr),1e-10)

In [22]:
en_sentences_translated_vectors = [normalize(avg_feature_vector(sentence, model=w2v_model, num_features=300, index2word_set=index2word_set)) for sentence in en_sentences_translated]
en_sentences_translated_vectors_2 = [normalize(avg_feature_vector(en_sentences_translated[i]+en_sentences_translated[i+1], model=w2v_model, num_features=300, index2word_set=index2word_set)) for i in range(len(en_sentences_translated)-1)]
en_sentences_translated_vectors_3 = [normalize(avg_feature_vector(en_sentences_translated[i]+en_sentences_translated[i+1]+en_sentences_translated[i+2], model=w2v_model, num_features=300, index2word_set=index2word_set)) for i in range(len(en_sentences_translated)-2)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [23]:
all_en_sentences_vectors = [normalize(avg_feature_vector(sentence, model=w2v_model, num_features=300, index2word_set=index2word_set)) for sentence in all_en_sentences]
all_en_sentences_vectors_2 = [normalize(avg_feature_vector(all_en_sentences[i]+all_en_sentences[i+1], model=w2v_model, num_features=300, index2word_set=index2word_set)) for i in range(len(all_en_sentences)-1)]
all_en_sentences_vectors_3 = [normalize(avg_feature_vector(all_en_sentences[i]+all_en_sentences[i+1]+all_en_sentences[i+2], model=w2v_model, num_features=300, index2word_set=index2word_set)) for i in range(len(all_en_sentences)-2)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [32]:
import numpy as np
i_1 = 0 
i_2 = 0 
threshold=0.05
m_mat = np.empty((3,300))
n_mat = np.empty((300,3))
matching_sentences = []
matching_translations = []
while i_1<len(en_sentences_translated) and i_2<len(all_en_sentences):
  m_mat[0,:] = en_sentences_translated_vectors[i_1]
  if i_1<len(en_sentences_translated)-1:
    m_mat[1,:] = en_sentences_translated_vectors_2[i_1]
  if i_1<len(en_sentences_translated)-2:
    m_mat[2,:] = en_sentences_translated_vectors_3[i_1]
  n_mat[:,0] = all_en_sentences_vectors[i_2]
  if i_2<len(all_en_sentences)-1:
    n_mat[:,1] = all_en_sentences_vectors_2[i_2]
  if i_2<len(all_en_sentences)-2:
    n_mat[:,2] = all_en_sentences_vectors_3[i_2]
  similarity = np.matmul(m_mat, n_mat)
  row_max = np.argmax(similarity[0,:])
  row_max_val = similarity[0,row_max]
  if similarity[0,0]+threshold>= row_max_val:
    row_max = 0
    row_max_val = similarity[0,0]
   
  col_max = np.argmax(similarity[:,0])
  col_max_val = similarity[col_max,0]
  if similarity[0,0]+threshold>= col_max_val:
    col_max = 0
    col_max_val = similarity[0,0]
   
  #print(similarity)

  if row_max_val>=col_max_val:
    matching_idxs = (1,row_max+1)
    matching_sentences.append(([en_sentences_translated[i_1]],all_en_sentences[i_2:(i_2+row_max+1)]))
    matching_translations.append(([de_sentences[i_1]],all_en_sentences[i_2:(i_2+row_max+1)]))
    #print((en_sentences_translated[i_1],all_en_sentences[i_2:(i_2+row_max+1)]))
    i_1 +=1
    i_2 += (row_max+1)
  else:
    matching_idxs = (col_max+1,1)
    matching_sentences.append((en_sentences_translated[i_1:(i_1+col_max+1)],[all_en_sentences[i_2]]))
    matching_translations.append((de_sentences[i_1:(i_1+col_max+1)],[all_en_sentences[i_2]]))
    #print((en_sentences_translated[i_1:(i_1+col_max+1)],all_en_sentences[i_2]))
    i_1 += (col_max+1)
    i_2 +=1
    

In [33]:
matching_sentences[:10]

[(['The trial, Franz Kafka 1 arrest, conversation with Mrs. Grubach, then Miss Brüstner Someone must have slandered Josef K., because without having done anything wrong he was arrested one morning.'],
  ['The Trial Franz Kafka Chapter One Arrest - Conversation with Mrs. Grubach - Then Miss Bürstner Someone must have been telling lies about Josef K., he knew he had done nothing wrong but, one morning, he was arrested.']),
 (["The cook of Mrs. Grubach, his room landlord, who brought him breakfast every day at around eight o'clock in the morning, did not come this time."],
  ["Every day at eight in the morning he was brought his breakfast by Mrs. Grubach's cook - Mrs. Grubach was his landlady - but today she didn't come."]),
 (['This had never happened before.'], ['That had never happened before.']),
 (['K. waited a little longer, saw from his pillow the old woman who lived opposite him, and who observed him with a curiosity quite unusual in her, but then, at the same time bewildered and 

In [34]:
matching_translations[:10]

[(['Der Prozess , Franz Kafka 1 Verhaftung , Gespräch mit Frau Grubach , dann Fräulein Brüstner Jemand mußte Josef K. verleumdet haben , denn ohne daß er etwas Böses getan hätte , wurde er eines Morgens verhaftet .'],
  ['The Trial Franz Kafka Chapter One Arrest - Conversation with Mrs. Grubach - Then Miss Bürstner Someone must have been telling lies about Josef K., he knew he had done nothing wrong but, one morning, he was arrested.']),
 (['Die Köchin der Frau Grubach , seiner Zimmervermieterin , die ihm jeden Tag gegen acht Uhr früh das Frühstück brachte , kam diesmal nicht .'],
  ["Every day at eight in the morning he was brought his breakfast by Mrs. Grubach's cook - Mrs. Grubach was his landlady - but today she didn't come."]),
 (['Das war noch niemals geschehen .'], ['That had never happened before.']),
 (['K. wartete noch ein Weilchen , sah von seinem Kopfkissen aus die alte Frau , die ihm gegenüber wohnte und die ihn mit einer an ihr ganz ungewöhnlichen Neugierde beobachtete , 

In [31]:
idx = 0
with open("%s%s_matched_en%s"%(base_path, book_name_base, book_name_ext), "w") as f:
  for item in matching_translations:
    f.write("%s|%s|%s\n"%(str(idx), "".join(item[0]), "".join(item[1])))
    idx+=1